In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy
import skimage
# import custom functions
import sys
sys.path.append("../../../../")
import utils
import os
from tqdm import tqdm
import torch
from skimage.io import imread
from scipy.io import loadmat
from skimage.measure import regionprops
from skimage.transform import resize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import graph


code does not include cropping images

In [2]:
# load data
df = pd.read_csv('../../../../../data/codex_murine/features_and_metadataV2.csv', index_col = 0)
df.head()

,Unnamed: 0,cellLabelInImage,cellSize,CD45,Ly6C,TCR,Ly6G,CD19,CD169,CD106,...,CD35,CD2135,CD44,nucl,NKp46,cluster.term,centroid_x,centroid_y,field_of_view,cluster.termV2
0,1,3,283,0.000000,316.262419,595.768897,49.077602,608.994748,322.609557,0.000000,...,217.914412,69.250551,0.000000,9100.068826,572.555576,Other,7.717314,240.809187,1,Vessel
1,4,7,215,0.000000,0.000000,134.472468,0.000000,1431.469046,73.665657,0.000000,...,1135.190597,96.309696,0.000000,15071.640583,0.000000,B,5.916279,1334.600000,1,Other
2,5,11,397,0.000000,372.035233,899.713809,261.276071,605.481258,1023.287300,1152.344425,...,488.606413,245.895189,236.229915,16381.046106,769.252953,Other,9.277078,1167.989924,1,Vessel
3,7,13,500,2966.989105,337.537437,329.896108,37.743424,671.873764,51.828695,49.372156,...,0.000000,0.000000,942.795851,2283.347509,312.484433,B,11.430000,1026.952000,1,B
4,9,15,838,3797.157518,665.627685,1938.002387,368.658711,2616.937947,1682.991647,3003.585919,...,1822.097852,392.223150,5141.479714,23181.488067,575.791169,Mph,10.233890,656.547733,1,Mph


In [3]:
data_dir = '/mnt/cloud1/sheng-projects/st_projects/spatial_clust/spatial-clust-scripts/ipynb/Bokai_reorg/spleen/data/'

#df.to_csv(os.path.join(data_dir, 'features_and_metadata.csv'))

In [4]:
# save out
# feature edge
# feature label
# cell nbhd
# spatial edge

features_names = [ # columns to use
        'CD45',
        'Ly6C',
        'TCR',
        'Ly6G',
        'CD19',
        'CD169',
        'CD106',
        'CD3',
        'CD1632',
        'CD8a',
        'CD90',
        'F480',
        'CD11c',
        'Ter119',
        'CD11b',
        'IgD',
        'CD27',
        'CD5',
        'CD79b',
        'CD71',
        'CD31',
        'CD4',
        'IgM',
        'B220',
        'ERTR7',
        'MHCII',
        'CD35',
        'CD2135',
        'CD44',
        'nucl',
        'NKp46'
    ]
features = df[features_names].to_numpy()
features = utils.center_scale(features)

res = 0.5
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge for snap-GNN (spatial)
spatial_edges = graph.get_spatial_edges(
    arr=locations, n_neighbors=15, verbose=True
)
np.save(os.path.join(data_dir, f'spatial_edges_0325.npy'), np.array(spatial_edges[:2]).T)


/home/sheng/miniconda3/envs/super-resolution/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculating cell neighborhood composition matrix...


In [5]:
## just check dims

print([cell_nbhd.shape, len(np.unique(feature_labels))])

[(53500, 15), 15]


Save out scaled version in case need use

In [6]:
features_names = [
        'CD45',
        'Ly6C',
        'TCR',
        'Ly6G',
        'CD19',
        'CD169',
        'CD106',
        'CD3',
        'CD1632',
        'CD8a',
        'CD90',
        'F480',
        'CD11c',
        'Ter119',
        'CD11b',
        'IgD',
        'CD27',
        'CD5',
        'CD79b',
        'CD71',
        'CD31',
        'CD4',
        'IgM',
        'B220',
        'ERTR7',
        'MHCII',
        'CD35',
        'CD2135',
        'CD44',
        'nucl',
        'NKp46'
    ]
features = df[features_names].to_numpy()
features = utils.center_scale(features)
np.save(os.path.join(data_dir, f'feature_scaled.npy'), features)

Below is custom code to get single cell morphology images saved, due to the formatting sturcture of this spleen dataset

In [1]:
## custom function made specifically for this dataset

def save_train_test_data(df, size, load_path, save_folder, trun_qr, shape_of_each_view=(1008, 1344),
                               channels=('CD45', 'nucl'), shape_of_views=(9, 7), pad = 1000, verbose = True):
    '''
    For all cells in the dataframe, locate it in the picture and crop the image of given size, padding 0 if needed
    Only select channels in the argument
    Save numpy array of size n_cells * size * size * n_channels
    
    
    Args:
    df is a pandas dataframe consisting of clean cells
    '''
    n_cell = df.shape[0]
    n_channel = len(channels)
    if verbose:
        print("Combining all views...", flush = True)
    ### make all view into one big view
    
    pad_image = np.zeros((shape_of_each_view[0]*shape_of_views[0], 
                          shape_of_each_view[1]*shape_of_views[1], n_channel))
    for view_j in range(shape_of_views[1]):
        for view_i in range(shape_of_views[0]):
            view = view_j * shape_of_views[0] + view_i + 1
            img = imread(
                    '{}/raw_images/focused_BALBc-1_X0{}_Y0{}.tif'.format(
                        load_path, str(view_j + 1), str(view_i + 1)
                    )
                )
            pad_image[view_i*shape_of_each_view[0]:(view_i+1)*shape_of_each_view[0],
                      view_j*shape_of_each_view[1]:(view_j+1)*shape_of_each_view[1],:] = select_useful_features(img, channels)
    
    if verbose:
        print("Done!", flush = True)
    
    zero_pad_entire_view = np.zeros((pad_image.shape[0]+2*pad, pad_image.shape[1]+2*pad, pad_image.shape[2]))
    zero_pad_entire_view[pad:pad_image.shape[0]+pad, pad:pad_image.shape[1]+pad, :] = pad_image
    #truncate = 7000
    truncate = np.quantile(pad_image, q = trun_qr, axis = (0, 1))
    truncate = truncate[None, None, :]
    
    print(f"the current truncation is {truncate}")
    
    zero_pad_entire_view[zero_pad_entire_view <= truncate] = 0
    zero_pad_entire_view[zero_pad_entire_view > truncate] = 1
    
    if verbose:
        print("Processing each cell...and saving!", flush = True)
    for i in tqdm(range(n_cell)):
        # process each cell
        center_x = df.iloc[i]["centroid_x"]
        center_y = df.iloc[i]["centroid_y"]
        cur_image = np.transpose(zero_pad_entire_view[(int(center_x)-size//2+pad):(int(center_x)+size//2+pad), 
                                               (int(center_y)-size//2+pad):(int(center_y)+size//2+pad), 
                                               :], (2, 0, 1)).astype(np.int8)
        assert(cur_image.shape == (2, size, size))
        if verbose:
            if i % 10000 == 1:
                plt.imshow(cur_image[0, :, :])
                plt.show()
                plt.imshow(cur_image[1, :, :])
                plt.show()
        
        np.save(file = os.path.join(save_folder, f"size{size}", "images", f"img_{i:05d}"), arr = cur_image)
        
    return
        

In [ ]:
load_path = '../../../data/codex_murine'
save_folder = os.path.join(load_path, "processed_data", "single_cell_images")

size = 1024
channels = ('CD45', 'nucl')
n_channels = len(channels)
if not os.path.exists(os.path.join(save_folder, f"size{size}", "images")):
    os.makedirs(os.path.join(save_folder, f"size{size}", "images"))

## trun_qr = 7000 is ~ quantile 0.9 in this dataset
save_train_test_data(df, size, load_path, save_folder, trun_qr = 7000, shape_of_each_view=(1008, 1344),
                                       channels=channels, shape_of_views=(9, 7), pad = 1000, verbose = True)